In [3]:
import pandas as pd

Import all the files

In [6]:
visits = pd.read_csv('visits.csv',
                     parse_dates=[1])
cart = pd.read_csv('cart.csv',
                   parse_dates=[1])
                   
checkout = pd.read_csv('checkout.csv',
                       parse_dates=[1])
purchase = pd.read_csv('purchase.csv',
                       parse_dates=[1])

Step 1: Inspect the DataFrames using `print` and `head`

In [36]:
print(visits.head(5))
print(cart.head(5))
print(checkout.head(5))
print(purchase.head(5))

                                user_id          visit_time
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00
                                user_id           cart_time
0  2be90e7c-9cca-44e0-bcc5-124b945ff168 2017-11-07 20:45:00
1  4397f73f-1da3-4ab3-91af-762792e25973 2017-05-27 01:35:00
2  a9db3d4b-0a0a-4398-a55a-ebb2c7adf663 2017-03-04 10:38:00
3  b594862a-36c5-47d5-b818-6e9512b939b3 2017-09-27 08:22:00
4  a68a16e2-94f0-4ce8-8ce3-784af0bbb974 2017-07-26 15:48:00
                                user_id       checkout_time
0  d33bdc47-4afa-45bc-b4e4-dbe948e34c0d 2017-06-25 09:29:00
1  4ac186f0-9954-4fea-8a27-c081e428e34e 2017-04-07 20:11:00
2  3c9c78a7-124a-4b77-8d2e-e1926e011e7d 2017-07-13 11:38:00
3  89fe330a-8966-4756-8f7c-3bdbcd47279a 

Step 2: Left merging visits and cart

In [19]:
visits_cart = visits.merge(cart, how = "left")
print(visits_cart.head(5))

                                user_id          visit_time  \
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00   

            cart_time  
0                 NaT  
1 2017-01-26 14:44:00  
2 2017-08-20 08:31:00  
3                 NaT  
4                 NaT  


Step 3: How long is `visits_cart`?

In [29]:
length_visits_cart = visits_cart.user_id.count()
print(length_visits_cart)

2000


Step 4: How many timestamps are null for `cart_time`?

In [25]:
null_cart_time = visits_cart["cart_time"].isnull().sum()
print(null_cart_time)

1652


Step 5: What percentage only visited?

In [71]:
only_visited = (float(null_cart_time)/float(length_visits_cart)) * 100
print(only_visited)

82.6


Step 6: What percentage placed a t-shirt in their cart but did not checkout?

In [103]:
cart_checkout = cart.merge(checkout, how = "left")

length_cart = cart_checkout.user_id.count()

not_checkout = cart_checkout["checkout_time"].isnull().sum()

only_cart = round(float(not_checkout) * 100 /float(length_cart), 2)

print(only_cart)

35.06


Step 7: Merge it all together

In [106]:
total_visits_data = visits_cart.merge(cart_checkout, how = "left").merge(purchase, how = "left")
print(total_visits_data.head())


                                user_id          visit_time  \
0  943647ef-3682-4750-a2e1-918ba6f16188 2017-04-07 15:14:00   
1  0c3a3dd0-fb64-4eac-bf84-ba069ce409f2 2017-01-26 14:24:00   
2  6e0b2d60-4027-4d9a-babd-0e7d40859fb1 2017-08-20 08:23:00   
3  6879527e-c5a6-4d14-b2da-50b85212b0ab 2017-11-04 18:15:00   
4  a84327ff-5daa-4ba1-b789-d5b4caf81e96 2017-02-27 11:25:00   

            cart_time       checkout_time       purchase_time  
0                 NaT                 NaT                 NaT  
1 2017-01-26 14:44:00 2017-01-26 14:54:00 2017-01-26 15:08:00  
2 2017-08-20 08:31:00                 NaT                 NaT  
3                 NaT                 NaT                 NaT  
4                 NaT                 NaT                 NaT  


Step 8: % of users who got to checkout but did not purchase

In [109]:
checkout_length = (~total_visits_data.checkout_time.isnull()).sum() 
print(checkout_length)
purchase_list = total_visits_data[(total_visits_data.purchase_time.isnull()) & (~total_visits_data.checkout_time.isnull())]
purchase_length = len(purchase_list)
purchase_not_completed = round(float(purchase_length) * 100 / float(checkout_length),2)
print(purchase_not_completed)

334
24.55


Step 9: check each part of the funnel, let's print all 3 of them again

In [112]:
print("{}% of the visitors didn't add any product to the cart".format(only_visited))
print("Among the visitors who added products to the cart, {}% did not checkout".format(only_cart))
print("Among those who added products to their cart, {}% did not finish de purchase process".format(purchase_not_completed))

82.6% of the visitors didn't add any product to the cart
Among the visitors who added products to the cart, 35.06% did not checkout
Among those who added products to their cart, 24.55% did not finish de purchase process


*The weakest part of the funnel is clearly getting a person who visited the site to add a tshirt to their cart. Once they've added a t-shirt to their cart it is fairly likely they end up purchasing it. A suggestion could be to make the add-to-cart button more prominent on the front page.*


Step 10: adding new column

In [128]:
total_visits_data["time_to_purchase"] = total_visits_data.purchase_time - total_visits_data.visit_time

Step 11: examine the results

In [130]:
print(total_visits_data.time_to_purchase.head(10))


0               NaT
1   0 days 00:44:00
2               NaT
3               NaT
4               NaT
5               NaT
6               NaT
7               NaT
8               NaT
9               NaT
Name: time_to_purchase, dtype: timedelta64[ns]


Step 12: average time to purchase

In [136]:
avg_purchase_time = total_visits_data.time_to_purchase.mean()
print("The average time from visiting the site to finish the purchase is {}".format(avg_purchase_time))

The average time from visiting the site to finish the purchase is 0 days 00:43:12.380952380
